In [50]:
import pandas as pd
import numpy as np

# Chargement des données

In [51]:
print("=== CHARGEMENT DES DONNÉES ===")
data = pd.read_csv("co2.csv")
print(f"Forme initiale : {data.shape}")
print(f"Nombre de lignes : {data.shape[0]}")
print(f"Nombre de colonnes : {data.shape[1]}")

=== CHARGEMENT DES DONNÉES ===
Forme initiale : (7385, 12)
Nombre de lignes : 7385
Nombre de colonnes : 12


In [52]:
data.head()

,Make,Model,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),CO2 Emissions(g/km)
0,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244


# 1. VÉRIFICATION DES VALEURS MANQUANTES

In [53]:
print("\n=== VALEURS MANQUANTES ===")
missing = data.isnull().sum()
print(missing[missing > 0] if missing.sum() > 0 else "Aucune valeur manquante détectée ✓")


=== VALEURS MANQUANTES ===
Aucune valeur manquante détectée ✓


# 2. SUPPRESSION DES DOUBLONS

In [54]:
print("\n=== SUPPRESSION DES DOUBLONS ===")
duplicates_count = data.duplicated().sum()
print(f"Nombre de doublons trouvés : {duplicates_count}")
if duplicates_count > 0:
    data.drop_duplicates(inplace=True, ignore_index=True)
    print(f"Doublons supprimés ✓")
    print(f"Nouvelle forme : {data.shape}")


=== SUPPRESSION DES DOUBLONS ===
Nombre de doublons trouvés : 1103
Doublons supprimés ✓
Nouvelle forme : (6282, 12)


# 3. NETTOYAGE DES NOMS DE COLONNES

In [55]:
print("\n=== NETTOYAGE DES NOMS DE COLONNES ===")
# Supprimer les espaces et standardiser
data.columns = data.columns.str.strip()
column_mapping = {
    'Engine Size(L)': 'Engine_Size_L',
    'Fuel Consumption City (L/100 km)': 'Fuel_City_L100km',
    'Fuel Consumption Hwy (L/100 km)': 'Fuel_Hwy_L100km',
    'Fuel Consumption Comb (L/100 km)': 'Fuel_Comb_L100km',
    'Fuel Consumption Comb (mpg)': 'Fuel_Comb_mpg',
    'CO2 Emissions(g/km)': 'CO2_Emissions_gkm',
    'Vehicle Class': 'Vehicle_Class',
    'Fuel Type': 'Fuel_Type'
}
data.rename(columns=column_mapping, inplace=True)
print("Nouveaux noms de colonnes :")
print(data.columns.tolist())


=== NETTOYAGE DES NOMS DE COLONNES ===
Nouveaux noms de colonnes :
['Make', 'Model', 'Vehicle_Class', 'Engine_Size_L', 'Cylinders', 'Transmission', 'Fuel_Type', 'Fuel_City_L100km', 'Fuel_Hwy_L100km', 'Fuel_Comb_L100km', 'Fuel_Comb_mpg', 'CO2_Emissions_gkm']


# 4. NETTOYAGE DES DONNÉES TEXTUELLES

In [56]:
print("\n=== NETTOYAGE DES DONNÉES TEXTUELLES ===")
# Supprimer les espaces en début/fin pour les colonnes texte
text_columns = ['Make', 'Model', 'Vehicle_Class', 'Transmission', 'Fuel_Type']
for col in text_columns:
    data[col] = data[col].str.strip()
print("Espaces supprimés dans les colonnes textuelles ✓")


=== NETTOYAGE DES DONNÉES TEXTUELLES ===
Espaces supprimés dans les colonnes textuelles ✓


# 5. VÉRIFICATION DES VALEURS ABERRANTES

In [57]:
print("\n=== VÉRIFICATION DES VALEURS ABERRANTES ===")

# Vérifier les valeurs négatives
numeric_cols = ['Engine_Size_L', 'Cylinders', 'Fuel_City_L100km', 
                'Fuel_Hwy_L100km', 'Fuel_Comb_L100km', 'Fuel_Comb_mpg', 
                'CO2_Emissions_gkm']

negative_values = False
for col in numeric_cols:
    neg_count = (data[col] < 0).sum()
    if neg_count > 0:
        print(f"⚠️ {col} : {neg_count} valeurs négatives trouvées")
        negative_values = True

if not negative_values:
    print("Aucune valeur négative détectée ✓")

# Vérifier les valeurs nulles dans les colonnes numériques
zero_values = False
for col in numeric_cols:
    zero_count = (data[col] == 0).sum()
    if zero_count > 0:
        print(f"ℹ️ {col} : {zero_count} valeurs à zéro")
        zero_values = True


=== VÉRIFICATION DES VALEURS ABERRANTES ===
Aucune valeur négative détectée ✓


# 6. VÉRIFICATION DE LA COHÉRENCE DES DONNÉES

In [58]:
print("\n=== VÉRIFICATION DE LA COHÉRENCE ===")

# La consommation combinée devrait être entre ville et autoroute
inconsistent = data[
    (data['Fuel_Comb_L100km'] < data['Fuel_Hwy_L100km']) | 
    (data['Fuel_Comb_L100km'] > data['Fuel_City_L100km'])
]
if len(inconsistent) > 0:
    print(f"⚠️ {len(inconsistent)} lignes avec consommation combinée incohérente")
else:
    print("Cohérence des consommations validée ✓")

inconsistent.head()


=== VÉRIFICATION DE LA COHÉRENCE ===
⚠️ 91 lignes avec consommation combinée incohérente


,Make,Model,Vehicle_Class,Engine_Size_L,Cylinders,Transmission,Fuel_Type,Fuel_City_L100km,Fuel_Hwy_L100km,Fuel_Comb_L100km,Fuel_Comb_mpg,CO2_Emissions_gkm
342,FORD,C-MAX HYBRID,FULL-SIZE,2.0,4,AV,X,5.6,6.4,6.0,47,138
415,FORD,FUSION HYBRID,MID-SIZE,2.0,4,AV,X,5.3,5.7,5.5,51,126
487,HONDA,ACCORD HYBRID,MID-SIZE,2.0,4,AV,X,4.7,4.9,4.8,59,110
659,LEXUS,CT 200h,COMPACT,1.8,4,AV,X,5.5,5.8,5.6,50,129
660,LEXUS,ES 300h,MID-SIZE,2.5,4,AV6,X,5.8,6.2,6.0,47,138


# 7. VÉRIFICATION DES CATÉGORIES

In [59]:
print("\n=== VALEURS UNIQUES PAR CATÉGORIE ===")
print(f"Marques (Make) : {data['Make'].nunique()}")
print(f"Classes de véhicules : {data['Vehicle_Class'].nunique()}")
print(f"Types de carburant : {data['Fuel_Type'].nunique()}")
print(f"  - {data['Fuel_Type'].value_counts().to_dict()}")


=== VALEURS UNIQUES PAR CATÉGORIE ===
Marques (Make) : 42
Classes de véhicules : 16
Types de carburant : 5
  - {'X': 3039, 'Z': 2765, 'E': 330, 'D': 147, 'N': 1}


# 8. VÉRIFICATION DES TYPES DE DONNÉES

In [60]:
print("\n=== TYPES DE DONNÉES ===")
print(data.dtypes)


=== TYPES DE DONNÉES ===
Make                  object
Model                 object
Vehicle_Class         object
Engine_Size_L        float64
Cylinders              int64
Transmission          object
Fuel_Type             object
Fuel_City_L100km     float64
Fuel_Hwy_L100km      float64
Fuel_Comb_L100km     float64
Fuel_Comb_mpg          int64
CO2_Emissions_gkm      int64
dtype: object


# 9. SAUVEGARDE DES DONNÉES NETTOYÉES

In [61]:
print("\n=== SAUVEGARDE ===")
output_file = 'Co2_Nettoye.csv'
data.to_csv(output_file, index=False)
print(f"✓ Données nettoyées sauvegardées dans '{output_file}'")
print(f"Forme finale : {data.shape}")


=== SAUVEGARDE ===
✓ Données nettoyées sauvegardées dans 'Co2_Nettoye.csv'
Forme finale : (6282, 12)


# 10. RÉSUMÉ FINAL

In [62]:
print("\n" + "="*50)
print("RÉSUMÉ DU NETTOYAGE")
print("="*50)
print(f"Lignes initiales : 7385")
print(f"Lignes finales : {data.shape[0]}")
print(f"Lignes supprimées : {7385 - data.shape[0]}")
print(f"Colonnes : {data.shape[1]}")
print("="*50)


RÉSUMÉ DU NETTOYAGE
Lignes initiales : 7385
Lignes finales : 6282
Lignes supprimées : 1103
Colonnes : 12
